Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [13]:
Nimply = namedtuple("Nimply", "row, num_objects")
N_ROWS = 5
MAX_LAST_ROW = N_ROWS*4

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [6]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [7]:
import numpy as np

def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


# My method
during the game it tries to find the `nim_sum` = 1 or near from it, without considering the 0 (it will take the 0 only if it is the only one solution) then it will make the move to win, that is a function that allows us to win.

This method provides to create random moves and select the best move(`nim_sum` = 1), if the random moves don't have a `nim_sum` = 1 it mutates the lower `nim_sum` and if two or more `rows` have the same `nim_sum` it considers the `row` that has the higher number of `objects`.

If there are one or two rows, the move that allows you to win will be made(`move_to_win`).
___
### my_strategy vs optimal:
- with move_to_win win rate around 75%-80%
- without move_to_win win rate around 63%

In [27]:
def PosNormal(mean, sigma, num_obj):
    x = np.random.normal(mean,sigma)
    return(x if x>=1 and x<num_obj else PosNormal(mean, sigma, num_obj))

def tweak(state: Nim, move: Nimply) -> Nimply:               #it takes in input Nim and Nimply and search a move near to its
    to_remove = int(PosNormal(move.num_objects, (state.rows[move.row]/2), (state.rows[move.row]+1)))#PosNormal(mean, sigma, number of objs)
    return Nimply(move.row, to_remove)

def modified_sort(num, max):
    if num == 0 :
        return max
    return num

def move_to_win(non_empty_rows, state: Nim) -> Nimply:
    if (len(non_empty_rows) == 1):
        if (state.rows[non_empty_rows[0]] == 1):
            return Nimply(non_empty_rows[0], 1)
        return Nimply(non_empty_rows[0], state.rows[non_empty_rows[0]]-1)
    if (len(non_empty_rows) == 2):
        if (state.rows[non_empty_rows[0]] == 1):
            return Nimply(non_empty_rows[1], state.rows[non_empty_rows[1]])
        if (state.rows[non_empty_rows[1]] == 1):
            return Nimply(non_empty_rows[0], state.rows[non_empty_rows[0]])
    return  None

def my_strategy(state: Nim) -> Nimply:
    non_empty_rows = [r for r, c in enumerate(state.rows) if c != 0]
    new_state = []          #new moves array

    move = move_to_win(non_empty_rows, state)           #this move to win provides to  do the last move that allows to win,
    if (move is not None) :                             #with this it has the 75%-80% win rate, without this it has the 65% vs the optimal
        print("----------------------------------")
        print("last move:", move)
        return move
    
    for _ in range(int(sum(state.rows[j] for j in non_empty_rows)/3)+1):
        tmp = deepcopy(state)
        r_var = pure_random(tmp)
        while r_var in [item[0] for item in new_state]:
            r_var = pure_random(tmp)
        tmp.nimming(r_var)
        new_state.append([r_var, nim_sum(tmp)])
    new_state.sort(key=lambda x: (modified_sort(x[1], MAX_LAST_ROW), state.rows[x[0].row]))
    print("----------------------------------")
    print("original:",new_state)
    if new_state[0][1] == 1:
        print("best solution was early found")
        return new_state[0][0]              #best solution early found
    if len(new_state) == 1:
        print("it is not possible to make other mutations")
        return new_state[0][0]              #best solution early found
    mutated = [new_state[0]]                #if it is not the best it provides to find the nearest one in the same row
    for _ in range(int(state.rows[new_state[0][0].row]/1.5)):
        tmp = deepcopy(state)
        r_var = tweak(tmp,new_state[0][0])
        while r_var in [item[0] for item in mutated]:
            r_var = tweak(tmp,new_state[0][0])
        tmp.nimming(r_var)
        mutated.append([r_var, nim_sum(tmp)])
    mutated.sort(key=lambda x: (modified_sort(x[1], MAX_LAST_ROW), -state.rows[x[0].row]))
    print("mutated:",mutated)
    return mutated[0][0]

## Oversimplified match

In [28]:
logging.getLogger().setLevel(logging.INFO)

strategy = (my_strategy, optimal)

nim = Nim(N_ROWS)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=7)
INFO:root:status: <1 3 5 7 2>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 4 2>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 5 4 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 3 2 4 1>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 2 4 1>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 3 0 4 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 3 0 3 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <0 3 0 0 1>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 0 0 1>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 0 won!


----------------------------------
original: [[Nimply(row=4, num_objects=5), 4], [Nimply(row=4, num_objects=4), 5], [Nimply(row=0, num_objects=1), 8], [Nimply(row=1, num_objects=1), 8], [Nimply(row=4, num_objects=1), 8], [Nimply(row=1, num_objects=2), 11], [Nimply(row=3, num_objects=5), 12], [Nimply(row=2, num_objects=3), 14], [Nimply(row=3, num_objects=6), 15]]
mutated: [[Nimply(row=4, num_objects=7), 2], [Nimply(row=4, num_objects=6), 3], [Nimply(row=4, num_objects=5), 4], [Nimply(row=4, num_objects=4), 5], [Nimply(row=4, num_objects=3), 6], [Nimply(row=4, num_objects=2), 7], [Nimply(row=4, num_objects=9), 0]]
----------------------------------
original: [[Nimply(row=4, num_objects=1), 2], [Nimply(row=4, num_objects=2), 3], [Nimply(row=3, num_objects=3), 4], [Nimply(row=2, num_objects=5), 4], [Nimply(row=3, num_objects=4), 5], [Nimply(row=3, num_objects=2), 7]]
mutated: [[Nimply(row=4, num_objects=1), 2], [Nimply(row=4, num_objects=2), 3]]
----------------------------------
original:

# TO SEE THE PERFORMANCE
if you run the code below I advise you to comment all print function in the function `my_strategy`

In [26]:
logging.getLogger().setLevel(logging.INFO)
p = [0,0]
ITERATION = 1000
strategy = (my_strategy, optimal)
for i in range(ITERATION):
    nim = Nim(5)
    player = 0
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    logging.info(f"status: Player {player} won!")
    p[player] += 1
print("player 0 wins =", p[0],"\t win rate =", p[0]*100/ITERATION)
print("player 1 wins =", p[1],"\t win rate =", p[1]*100/ITERATION)


INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!


INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 0 won!
INFO:root:status: Player 1 won!
INFO:roo

player 0 wins = 791 	 win rate = 79.1
player 1 wins = 209 	 win rate = 20.9
